In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 50
pd.options.display.max_columns = 30
# !!!!!
# This file only consist of the part where I tidy up the data, the visualization part is in another file
# !!!!!

In [13]:
cards =pd.read_csv("choice1.csv")
#cards = cards.drop(["Image_name", "Other names", "Password","Ritual required","Ritual Monster required", 
                    #"Other names (Japanese)", "Summoned by the effect of"], axis = 1)

cards1 = cards
cards

,Image_name,Card-set,Card_number,Rarity,Card_name,Other names,Card type,Attribute,Types,Level,ATK / DEF,Password,Status,Effect types,Ritual required,Property,Ritual Monster required,Pendulum Scale,Rank,Link Arrows,ATK / LINK,Other names (Japanese),Summoned by the effect of
0,MRD-116 R.png,Metal Raiders,MRD-116,Rare,Witch of the Black Forest,NaN,Monster,DARK,Spellcaster / Effect,4.0,1100 / 1200,78010363,"['Unlimited (OCG)', 'Unlimited (TCG)']","['Trigger', 'Condition']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PHSW-EN014 C.png,Photon Shockwave,PHSW-EN014,Common,Plasma Ball,NaN,Monster,LIGHT,Thunder / Effect,3.0,900 / 900,92595545,"['Unlimited (OCG)', 'Unlimited (TCG)']","['Continuous', 'Trigger']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ROTD-EN030 UR.png,Rise of the Duelist,ROTD-EN030,Ultra Rare,Koa'ki Meiru Supplier,NaN,Monster,EARTH,Rock / Effect,4.0,1400 / 1600,80839052,"['Unlimited (OCG)', 'Unlimited (TCG)']","['Trigger', 'Trigger', 'Condition']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PGD-093 R.png,Pharaonic Guardian,PGD-093,Rare,Coffin Seller,NaN,Trap,NaN,NaN,NaN,NaN,65830223,"['Unlimited (OCG)', 'Unlimited (TCG)', 'Unlimi...",['Trigger-like'],NaN,Continuous,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RIRA-EN060 C.png,Rising Rampage,RIRA-EN060,Common,"Elborz, the Sacred Lands of Simorgh",NaN,Spell,NaN,NaN,NaN,NaN,92223430,"['Unlimited (OCG)', 'Unlimited (TCG)']","['Continuous-like', 'Condition', 'Ignition-lik...",NaN,Field,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8368,TDIL-EN004 C.png,The Dark Illusion,TDIL-EN004,Common,Performapal Inflater Tapir,NaN,Monster,WIND,Beast / Effect,6.0,1000 / 2000,71985676,"['Unlimited (OCG)', 'Unlimited (TCG)']","['Quick', 'Trigger', 'Condition']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8369,LODT-EN039 R.png,Light of Destruction,LODT-EN039,Rare,"Simorgh, Bird of Ancestry",NaN,Monster,WIND,Winged Beast / Effect,8.0,2900 / 2000,86489182,"['Unlimited (OCG)', 'Unlimited (TCG)']","['Continuous', 'Trigger', 'Unclassified']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8370,DAMA-EN066 ScR.png,Dawn of Majesty,DAMA-EN066,Secret Rare,Ready Fusion,NaN,Spell,NaN,NaN,NaN,NaN,63854005,"['Unlimited (OCG)', 'Unlimited (TCG)']","['Cost', 'Effect', 'Condition']",NaN,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8371,MACR-EN079 ScR.png,Maximum Crisis,MACR-EN079,Secret Rare,Unending Nightmare,NaN,Trap,NaN,NaN,NaN,NaN,69452756,"['Unlimited (OCG)', 'Unlimited (TCG)']","['Quick-like', 'Condition']",NaN,Continuous,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
cards1 = cards.rename(columns={"ATK / DEF": "ATK_DEF"})

atk_def = pd.DataFrame(cards1["ATK_DEF"].astype(str).str.split().values.tolist())
atk_def = atk_def.drop([1], axis = 1) # Drop "/" from str.split()
atk_def = atk_def.rename(columns = {0 : "ATK", 2 : "DEF"}) #Split Atk and Def into 2 dif col

cards1["ATK"] = atk_def["ATK"]
cards1["DEF"] = atk_def["DEF"]
#cards1 = cards1.fillna(value=np.nan) for OCD reasons
cards1 = cards1.drop(["ATK_DEF"], axis = 1)

ban_list1 = cards1["Status"].apply(lambda x: x.replace("[","").replace("]","")
                                   .replace("'", "").replace("(TCG)", "").replace("(TCG Advanced)",""))
ban_list1 = pd.DataFrame(ban_list1)
ban_list1

ban_list = pd.DataFrame(ban_list1["Status"].str.split(pat = ",").values.tolist())   # acutally works, need to remove [] and ''
ban_list = ban_list.rename(columns = {0 : "OCG Banlist", 1 : "TCG Banlist", 2 : "Speed Duel Banlist"})
ban_list
#cards1["OCG Banlist"] = ban_list["OCG Banlist"]
#cards1["Speed Duel Banlist"] = ban_list["Speed Duel Banlist"] Focus on TCG, since in UK :)
cards1["TCG_Banlist"] = ban_list["TCG Banlist"]
cards1 = cards1.drop(["Status"], axis = 1) #Drop old banlist
cards1 = cards1.dropna(subset=["TCG_Banlist"]) # There are cards which is only in OCG, not many but it exist
cards1 = cards1.drop_duplicates(subset=["Card_number"])
cards1 = cards1.reset_index(drop = True) #When removing row, index are not automatically updated, must reset


In [4]:
cards1 = cards1.rename(columns={"Effect types": "Effect_types"})
types = pd.DataFrame(cards1["Types"].astype(str).str.split(pat = "/").values.tolist())
types = types.rename(columns={0: "Monster_type", 1: "Type_1", 2: "Type_2", 3: "Type_3"})

cards1["Monster_type"] = types["Monster_type"]
cards1["Type_1"] = types["Type_1"]
cards1["Type_2"] = types["Type_2"]
cards1["Type_3"] = types["Type_3"]
cards1 = cards1.drop(cards1.columns[6], axis = 1) # Drop old types


In [5]:
effect = cards1["Effect_types"].astype(str).apply(lambda x: x.replace("[","").replace("]","")
                                   .replace("'", ""))
effect = pd.DataFrame(effect)
effect1 = pd.DataFrame(effect["Effect_types"].str.split(pat = ",").values.tolist())
effect # lot of cards effect types, yikes
effect1 = effect1.rename(columns={0: "E_type1", 1: "E_type2", 2: "E_type3", 3: "E_type4", 4: "E_type5", 5: "E_type6",
                                6 : "E_type7", 7: "E_type8", 8: "E_type9", 9: "E_type10"})
effect1

,E_type1,E_type2,E_type3,E_type4,E_type5,E_type6,E_type7,E_type8,E_type9,E_type10
0,Trigger,Condition,None,None,None,None,None,None,None,None
1,Continuous,Trigger,None,None,None,None,None,None,None,None
2,Trigger,Trigger,Condition,None,None,None,None,None,None,None
3,Trigger-like,None,None,None,None,None,None,None,None,None
4,Continuous-like,Condition,Ignition-like,Ignition-like,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
8322,Quick,Trigger,Condition,None,None,None,None,None,None,None
8323,Continuous,Trigger,Unclassified,None,None,None,None,None,None,None
8324,Cost,Effect,Condition,None,None,None,None,None,None,None
8325,Quick-like,Condition,None,None,None,None,None,None,None,None


In [6]:
cards1 = cards1.rename(columns={"ATK / LINK": "ATK_LINK"})
atk_link = pd.DataFrame(cards1["ATK_LINK"].astype(str).str.split(pat = " / ").values.tolist())
atk_link = atk_link.rename(columns={0: "ATK_link", 1: "Link_no"}) # Try to merge ?
cards1 = cards1.astype(str)
atk_link = atk_link.astype(str)


In [7]:

yo = cards1["ATK"] + atk_link["ATK_link"]
yo = pd.DataFrame(yo)
yo2 = yo[0].astype(str).apply(lambda x: x.replace("nan","").replace("a",""))
yo2 = pd.DataFrame(yo2)
yo2 = yo2.replace(r'\s+( +\.)|#',np.nan,regex=True).replace('',np.nan)
yo2
cards1 = cards1.drop(["ATK", "ATK_LINK", "Link Arrows", "Pendulum Scale"], axis = 1) # replace old ATK with ATK + link
cards1["ATK"] = yo2[0]
cards1["Link_no"] = atk_link["Link_no"]




In [8]:
cards1 = pd.concat([cards1, effect1], axis = 1)
cards1 = cards1.drop("Effect_types", axis = 1 )


In [9]:

cards1 = cards1.rename(columns = {"Monster_type" : "M_type1", "Type_1": "M_type2", "Type_2": "M_type3", "Type_3": "M_type4"})
cards1
new_lv = cards1["Level"] + cards1["Rank"]
new_lv = new_lv.str.replace("nan", "")
new_lv = new_lv.replace(r'\s+( +\.)|#',np.nan,regex=True).replace('',np.nan)
new_lv = pd.DataFrame(new_lv)

cards1 = cards1.drop("Level", axis = 1 )

In [10]:
#rearrange ?
cards1["Level"] = new_lv
cards1 = cards1.drop("Rank", axis = 1 )
cols = cards1.columns.tolist()
cols = ['Card-set',
 'Card_number',
 'Rarity',
 'Card_name',
 'Card type',
 'Attribute',
 'ATK',
 'DEF',
 'Level',
 'Link_no',
 'Property',
 'M_type1',
 'M_type2',
 'M_type3',
 'M_type4',
 'E_type1',
 'E_type2',
 'E_type3',
 'E_type4',
 'E_type5',
 'E_type6',
 'E_type7',
 'E_type8',
 'E_type9',
 'E_type10',
 'TCG_Banlist']

cards1 = cards1[cols]
cards1

,Card-set,Card_number,Rarity,Card_name,Card type,Attribute,ATK,DEF,Level,Link_no,Property,M_type1,M_type2,M_type3,M_type4,E_type1,E_type2,E_type3,E_type4,E_type5,E_type6,E_type7,E_type8,E_type9,E_type10,TCG_Banlist
0,Metal Raiders,MRD-116,Rare,Witch of the Black Forest,Monster,DARK,1100,1200,4.0,None,nan,Spellcaster,Effect,None,None,Trigger,Condition,None,None,None,None,None,None,None,None,Unlimited
1,Photon Shockwave,PHSW-EN014,Common,Plasma Ball,Monster,LIGHT,900,900,3.0,None,nan,Thunder,Effect,None,None,Continuous,Trigger,None,None,None,None,None,None,None,None,Unlimited
2,Rise of the Duelist,ROTD-EN030,Ultra Rare,Koa'ki Meiru Supplier,Monster,EARTH,1400,1600,4.0,None,nan,Rock,Effect,None,None,Trigger,Trigger,Condition,None,None,None,None,None,None,None,Unlimited
3,Pharaonic Guardian,PGD-093,Rare,Coffin Seller,Trap,nan,NaN,None,NaN,None,Continuous,nan,None,None,None,Trigger-like,None,None,None,None,None,None,None,None,None,Unlimited
4,Rising Rampage,RIRA-EN060,Common,"Elborz, the Sacred Lands of Simorgh",Spell,nan,NaN,None,NaN,None,Field,nan,None,None,None,Continuous-like,Condition,Ignition-like,Ignition-like,None,None,None,None,None,None,Unlimited
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8322,The Dark Illusion,TDIL-EN004,Common,Performapal Inflater Tapir,Monster,WIND,1000,2000,6.0,None,nan,Beast,Effect,None,None,Quick,Trigger,Condition,None,None,None,None,None,None,None,Unlimited
8323,Light of Destruction,LODT-EN039,Rare,"Simorgh, Bird of Ancestry",Monster,WIND,2900,2000,8.0,None,nan,Winged Beast,Effect,None,None,Continuous,Trigger,Unclassified,None,None,None,None,None,None,None,Unlimited
8324,Dawn of Majesty,DAMA-EN066,Secret Rare,Ready Fusion,Spell,nan,NaN,None,NaN,None,Normal,nan,None,None,None,Cost,Effect,Condition,None,None,None,None,None,None,None,Unlimited
8325,Maximum Crisis,MACR-EN079,Secret Rare,Unending Nightmare,Trap,nan,NaN,None,NaN,None,Continuous,nan,None,None,None,Quick-like,Condition,None,None,None,None,None,None,None,None,Unlimited


In [11]:
cards1 = cards1.rename(columns = {"Card-set" : "Card_set", "Card type" : "Card_type"})
cards1

,Card_set,Card_number,Rarity,Card_name,Card_type,Attribute,ATK,DEF,Level,Link_no,Property,M_type1,M_type2,M_type3,M_type4,E_type1,E_type2,E_type3,E_type4,E_type5,E_type6,E_type7,E_type8,E_type9,E_type10,TCG_Banlist
0,Metal Raiders,MRD-116,Rare,Witch of the Black Forest,Monster,DARK,1100,1200,4.0,None,nan,Spellcaster,Effect,None,None,Trigger,Condition,None,None,None,None,None,None,None,None,Unlimited
1,Photon Shockwave,PHSW-EN014,Common,Plasma Ball,Monster,LIGHT,900,900,3.0,None,nan,Thunder,Effect,None,None,Continuous,Trigger,None,None,None,None,None,None,None,None,Unlimited
2,Rise of the Duelist,ROTD-EN030,Ultra Rare,Koa'ki Meiru Supplier,Monster,EARTH,1400,1600,4.0,None,nan,Rock,Effect,None,None,Trigger,Trigger,Condition,None,None,None,None,None,None,None,Unlimited
3,Pharaonic Guardian,PGD-093,Rare,Coffin Seller,Trap,nan,NaN,None,NaN,None,Continuous,nan,None,None,None,Trigger-like,None,None,None,None,None,None,None,None,None,Unlimited
4,Rising Rampage,RIRA-EN060,Common,"Elborz, the Sacred Lands of Simorgh",Spell,nan,NaN,None,NaN,None,Field,nan,None,None,None,Continuous-like,Condition,Ignition-like,Ignition-like,None,None,None,None,None,None,Unlimited
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8322,The Dark Illusion,TDIL-EN004,Common,Performapal Inflater Tapir,Monster,WIND,1000,2000,6.0,None,nan,Beast,Effect,None,None,Quick,Trigger,Condition,None,None,None,None,None,None,None,Unlimited
8323,Light of Destruction,LODT-EN039,Rare,"Simorgh, Bird of Ancestry",Monster,WIND,2900,2000,8.0,None,nan,Winged Beast,Effect,None,None,Continuous,Trigger,Unclassified,None,None,None,None,None,None,None,Unlimited
8324,Dawn of Majesty,DAMA-EN066,Secret Rare,Ready Fusion,Spell,nan,NaN,None,NaN,None,Normal,nan,None,None,None,Cost,Effect,Condition,None,None,None,None,None,None,None,Unlimited
8325,Maximum Crisis,MACR-EN079,Secret Rare,Unending Nightmare,Trap,nan,NaN,None,NaN,None,Continuous,nan,None,None,None,Quick-like,Condition,None,None,None,None,None,None,None,None,Unlimited


In [12]:
cards1 = cards1.replace(r'\s+( +\.)|#',np.nan,regex=True).replace('',np.nan)
card1 = cards1.mask(cards1 == '')
cards1.to_csv("Card_tidy.csv")